# importing libraries

In [1]:
!pip install yfinance

     |████████████████████████████████| 6.3MB 7.2MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=9ebd4fd64c8dd4889217a95da98f93f331ee6282ade6ab71371018be5e351dd3
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.4MB/s 
     |████████████████████████████████| 3.3MB 29.5MB/s 
     |████████████████████████████████| 901kB 39.6MB/s 


In [3]:
import tweepy
import csv
import pandas as pd
import random
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import matplotlib.pyplot as mlpt
import yfinance as yf
import plotly.graph_objects as go
from transformers import pipeline
SentimentAnalysis=pipeline("sentiment-analysis")

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


# defining functions

In [4]:
def value_generate(item1,item2):
    if item1=='NEGATIVE':
        return -1*item2
    else:
        return item2
    
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

# extracting tweets

In [5]:
consumer_key    = '3jmA1BqasLHfItBXj3KnAIGFB'
consumer_secret = 'imyEeVTctFZuK62QHmL1I0AUAMudg5HKJDfkx0oR7oFbFinbvA'

access_token  = '265857263-pF1DRxgIcxUbxEEFtLwLODPzD3aMl6d4zOKlMnme'
access_token_secret = 'uUFoOOGeNJfOYD3atlcmPtaxxniXxQzAU4ESJLopA1lbC'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

fetch_tweets=tweepy.Cursor(api.search, q="#unitedAIRLINES",count=100, lang ="en",since="2018-9-13", tweet_mode="extended").items()
data=pd.DataFrame(data=[[tweet_info.created_at.date(),tweet_info.full_text]for tweet_info in fetch_tweets],columns=['Date','Tweets'])
data.to_csv("Tweets.csv")
cdata=data
cdata['Tweets']=cdata['Tweets'].apply(lambda x:re.sub(r'[^\w]',' ',x))
cdata['Tweets']=cdata['Tweets'].apply(lambda x:re.sub(r'[^a-zA-Z]',' ',x))
cdata['token']=cdata['Tweets'].apply(lambda x:x.split())
cdata['token']=cdata['token'].apply(lambda x : [i.lower() for i in x ] )
cdata['token']=cdata['token'].apply(lambda x : [i for i in x if not i.startswith('x')] )
redundant_words=['rt','nrt']
cdata['token']=cdata['token'].apply(lambda x : [i for i in x if i not in redundant_words] )
cdata['token']=cdata['token'].apply(lambda x : [i for i in x if len(i)>3])
cdata['Tweets']=cdata['token'].apply(lambda x:' '.join(x))
cdata=cdata.drop(columns='token')
cdata['Tweets']=cdata.groupby(['Date'])['Tweets'].transform(lambda x: ','.join(x))
cdata=cdata.drop_duplicates().reset_index(drop=True)
cdata['Date']=cdata['Date'].apply(lambda x: str(x)[0:10])

In [6]:
cdata

,Date,Tweets
0,2021-05-02,sure watch first ever video continental https ...
1,2021-05-01,delta delta seriously americanairlines regular...
2,2021-04-30,elizabetb tried over from kosovo through vienn...
3,2021-04-29,librarian spring united airlines timetable pos...
4,2021-04-28,phillipsa please support companies that fight ...
5,2021-04-27,believe these overhead luggage space rendered ...
6,2021-04-26,someone explain purpose social distancing airp...
7,2021-04-25,well unitedairlines unitedmileageprogram used ...


# importing data from yahoo finance

In [7]:
UAL = yf.Ticker("UAL")
data=UAL.history(period="1mo")
data=data.reset_index()
data=data[['Date','Open']]
data['Date']=data['Date'].apply(lambda x: str(x)[0:10])

In [8]:
data

,Date,Open
0,2021-04-01,57.660000
1,2021-04-05,59.340000
2,2021-04-06,59.880001
3,2021-04-07,59.790001
4,2021-04-08,59.139999
5,2021-04-09,59.020000
6,2021-04-12,57.570000
7,2021-04-13,54.410000
8,2021-04-14,57.459999
9,2021-04-15,57.790001


# processing of data

In [9]:
merged_data=pd.merge(cdata,data,how='left')
merged_data=merged_data.fillna(merged_data['Open'].mean())
merged_data['Open'] = merged_data['Open'].apply(np.int64)

In [10]:
merged_data

,Date,Tweets,Open
0,2021-05-02,sure watch first ever video continental https ...,53
1,2021-05-01,delta delta seriously americanairlines regular...,53
2,2021-04-30,elizabetb tried over from kosovo through vienn...,53
3,2021-04-29,librarian spring united airlines timetable pos...,54
4,2021-04-28,phillipsa please support companies that fight ...,53
5,2021-04-27,believe these overhead luggage space rendered ...,53
6,2021-04-26,someone explain purpose social distancing airp...,54
7,2021-04-25,well unitedairlines unitedmileageprogram used ...,53


# logic:
## 1. find the std deviation of the price
## 2. keep a weightage of 40 percent for the news impact 

In [11]:
std_deviation= merged_data.iloc[-4:,2].std()
merged_data['prev_Open']=merged_data['Open'].shift(-1)
merged_data['std_deviation']=std_deviation

In [12]:
merged_data

,Date,Tweets,Open,prev_Open,std_deviation
0,2021-05-02,sure watch first ever video continental https ...,53,53.0,0.5
1,2021-05-01,delta delta seriously americanairlines regular...,53,53.0,0.5
2,2021-04-30,elizabetb tried over from kosovo through vienn...,53,54.0,0.5
3,2021-04-29,librarian spring united airlines timetable pos...,54,53.0,0.5
4,2021-04-28,phillipsa please support companies that fight ...,53,53.0,0.5
5,2021-04-27,believe these overhead luggage space rendered ...,53,54.0,0.5
6,2021-04-26,someone explain purpose social distancing airp...,54,53.0,0.5
7,2021-04-25,well unitedairlines unitedmileageprogram used ...,53,NaN,0.5


In [13]:
merged_data['label']=merged_data['Tweets'].apply(lambda x:SentimentAnalysis(str(x)[0:200])[0]['label'])
merged_data['value']=merged_data['Tweets'].apply(lambda x:SentimentAnalysis(str(x)[0:200])[0]['score'])
merged_data['final_value']=merged_data.apply(lambda x:value_generate(x['label'],x['value']),axis=1)

In [14]:
merged_data['pred_open_price']=merged_data['prev_Open']+merged_data['std_deviation']*merged_data['final_value']

In [15]:
merged_data

,Date,Tweets,Open,prev_Open,std_deviation,label,value,final_value,pred_open_price
0,2021-05-02,sure watch first ever video continental https ...,53,53.0,0.5,NEGATIVE,0.527247,-0.527247,52.736376
1,2021-05-01,delta delta seriously americanairlines regular...,53,53.0,0.5,NEGATIVE,0.917919,-0.917919,52.541040
2,2021-04-30,elizabetb tried over from kosovo through vienn...,53,54.0,0.5,NEGATIVE,0.991895,-0.991895,53.504053
3,2021-04-29,librarian spring united airlines timetable pos...,54,53.0,0.5,NEGATIVE,0.948994,-0.948994,52.525503
4,2021-04-28,phillipsa please support companies that fight ...,53,53.0,0.5,POSITIVE,0.992772,0.992772,53.496386
5,2021-04-27,believe these overhead luggage space rendered ...,53,54.0,0.5,NEGATIVE,0.999577,-0.999577,53.500212
6,2021-04-26,someone explain purpose social distancing airp...,54,53.0,0.5,NEGATIVE,0.994297,-0.994297,52.502851
7,2021-04-25,well unitedairlines unitedmileageprogram used ...,53,NaN,0.5,NEGATIVE,0.997453,-0.997453,NaN


# results

In [16]:
print('the mape value is:',round(mape(merged_data['Open'][0:-1],merged_data['pred_open_price'][0:-1])))

the mape value is: 1


# actual vs prediction

In [17]:

N = 9
random_x = np.linspace(0, 1, N)
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=merged_data['Open'][0:-1],
                    mode='lines',
                    name='actual'))
fig.add_trace(go.Scatter(x=random_x, y=merged_data['pred_open_price'][0:-1],
                    mode='lines+markers',
                    name='predicted'))

fig.show()